<a href="https://colab.research.google.com/github/habbang8838/BDAA_treasure_in_data/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%86%8D_%EB%B3%B4%EB%AC%BC%EC%B0%BE%EA%B8%B0_%ED%8C%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 데이터 전처리 하기

In [19]:
#먼저 필요한 모듈을 import해줍니다

import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
# 시각화 과정에서 한글 깨짐 방지 코드입니다.

from matplotlib import font_manager, rc
import matplotlib
import platform
import matplotlib.pyplot as plt

%matplotlib inline 

if platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family = font_name)
else:
    rc('font',family='AppleGothic')
    
matplotlib.rcParams['axes.unicode_minus']=False
%matplotlib inline

In [21]:
# 데이터프레임은 월별로 시트가 여러개이므로, csv 파일로 불러오는 것이 불가능하여, 리스트에 요소별로 데이터프레임을 추가하고, concat하는 방식으로 하였습니다.
df = []
for i in range(9):
    df.append(pd.read_excel('BDAA_1회 내부 공모전 데이터.xlsx', sheet_name=i))

FileNotFoundError: [Errno 2] No such file or directory: 'BDAA_1회 내부 공모전 데이터.xlsx'

In [ ]:
df = pd.concat([df[i] for i in range(9)], axis = 0)

In [ ]:
df.head()

In [ ]:
#날짜 중 '일'이 역순으로 되어 있어 순서대로 정렬하였습니다.
df = df.sort_values('주문접수일시')
df

## 1) 분석에 필요하지 않은 칼럼 삭제하기

In [ ]:
df.info()

In [ ]:
df.isna().sum()

- 엑셀 파일 분석 결과 '진행상태 (약식)', '배송회차', '모델번호'는 일절 쓰지 않는 칼럼(총 데이터가 102077개인데 na가 102077개),
- '롯데멤버스오프라인고객번호', '롯데맴버스온라인고객번호','회원번호'는 6~9월에만 사용했던 칼럼이기에,
- 이 변수들로는 1 ~ 9월의 주문 경향을 분석하기에는 다소 어려움이 있어, 칼럼을 삭제하도록 하겠습니다.
- '추가옵션'이라는 칼럼 또한 모든 값이 []라는 널 값이기에, 삭제하였습니다.

In [ ]:
df = df.drop(['진행상태 (약식)', '모델번호', '롯데맴버스오프라인고객번호', '롯데맴버스온라인고객번호', '회원번호'], axis = 1)

- 칼럼이 너무 많아서, 단일 값이거나, 널값인 칼럼은 제거하도록 하겠습니다.

In [ ]:
for i in range(len(df.columns)):
    print(df.columns[i], len(df[df.columns[i]].unique()))

In [ ]:
print(df['배송수단'].unique())
print(df['배송상품유형'].unique())
print(df['배송처리유형'].unique())
print(df['배송예약구분'].unique())
print(df['배송회차'].unique())
print(df['추가옵션'].unique())
print(df['주문판매유형'].unique())
print(df['상품유형'].unique())

In [ ]:
# 브랜드명의 경우 전부 값이 씨제이 제일제당으로 같으므로 제거해주겠습니다.
df['브랜드'].value_counts()

In [ ]:
df = df.drop(['배송수단', '배송상품유형', '배송처리유형', '배송예약구분', '배송회차', '추가옵션', '주문판매유형', '상품유형', '브랜드'], axis=1)

In [ ]:
# 불필요한 칼럼을 제거하고, 27개로 남긴 상태입니다.
len(df.columns)

In [ ]:
df['예외처리구분'].unique()

In [ ]:
df[df['예외처리구분'] == '발송약정']

In [ ]:
print(df['유형'].unique())
print(df['구매확정'].unique())
print(df['예외처리구분'].unique())

In [ ]:
df[df['구매확정일시'].isna()]

## 2)주문완료된 데이터만 추출하기

In [ ]:
# 반품 및 교환 항목, 구매확정이 되지 않은 항목, 
# 예외처리구분(구매확정이 되었어도 이후 취소된 항목, 미수령되어 철회된 항목 등)은 제외하고, 
# 확실하게 주문이 성공된 상품에 대해서만 df_ordered라는 데이터프레임에 넣어주도록 하겠습니다.

df_ordered = df[df['유형'] == '주문']
df_ordered = df_ordered[df_ordered['구매확정'] == '구매확정']
df_ordered = df_ordered[df_ordered['예외처리구분'].isna()]

In [ ]:
df_ordered

## 3) 필요한 변수 생성하기

In [ ]:
#판매액 변수 생성
df_ordered['판매액'] = df_ordered['판매가'] * df_ordered['수량']

In [ ]:
#데이트 형식으로 변경
df_ordered['주문접수일시'] = pd.to_datetime(df_ordered['주문접수일시'], format = "%Y-%m-%d %H:%M:%S")

In [ ]:
#월별 파악하기 위해 변수설정
df_ordered['주문월'] = df_ordered['주문접수일시'].dt.month

In [ ]:
df_ordered = df_ordered.reset_index()

In [ ]:
df_ordered = df_ordered.drop(['index'], axis=1)

In [ ]:
df_ordered.head()

In [ ]:
for i in range(len(df_ordered.columns)):
    print(df_ordered.columns[i], len(df_ordered[df_ordered.columns[i]].unique()))

In [ ]:
# 주문 확정된 상품에서 더 이상 필요하지 않은 변수를 삭제하였습니다.
df_ordered.drop(['유형', '배송회수구분', '구매확정', '예외처리구분'], axis = 1, inplace=True)

In [ ]:
# 주문 확정된 상품 파일로 저장
# df_ordered.to_csv('CJ제일제당 주문 데이터(전처리).csv')

In [ ]:
df = pd.read_csv(path+'CJ제일제당 주문 데이터(전처리).csv')

# 2. 월별로 구매 특징 분석하기

In [ ]:
df['주문접수일시'] = pd.to_datetime(df['주문접수일시'], format = "%Y-%m-%d %H:%M:%S")

In [ ]:
# 시간대 파악을 위한 변수 생성
df['시간대'] = df['주문접수일시'].dt.hour
df['시간대 대분류'] = '21 ~ 00'
df.loc[(df['시간대'] >= 0) & (df['시간대'] < 3),'시간대 대분류'] = '00 ~ 03'
df.loc[(df['시간대'] >= 3) & (df['시간대'] < 6),'시간대 대분류'] = '03 ~ 06'
df.loc[(df['시간대'] >= 6) & (df['시간대'] < 9),'시간대 대분류'] = '06 ~ 09'
df.loc[(df['시간대'] >= 9) & (df['시간대'] < 12),'시간대 대분류'] = '09 ~ 12'
df.loc[(df['시간대'] >= 12) & (df['시간대'] < 15),'시간대 대분류'] = '12 ~ 15'
df.loc[(df['시간대'] >= 15) & (df['시간대'] < 18),'시간대 대분류'] = '15 ~ 18'
df.loc[(df['시간대'] >= 18) & (df['시간대'] < 21),'시간대 대분류'] = '18 ~ 21'

In [ ]:
df['유입채널'].value_counts().head(20)

In [ ]:
#유입채널 변수가 너무 다양하고 손쉽게 파악이 되지 않아 많이 나오는 변수대로 키워드를 묶어주었습니다.
df['유입채널 대분류'] = '기타 및 이벤트 광고'
df.loc[((df['유입채널'] == '유실 채널상세') | (df['유입채널'] == '디폴트 채널상세')), '유입채널 대분류'] = 'CJ 앱/사이트'
df.loc[df['유입채널'].str.contains('가격비교'), '유입채널 대분류'] = '가격비교'
df.loc[df['유입채널'].str.contains('모바일'), '유입채널 대분류'] = '모바일 공유'
df.loc[df['유입채널'].str.contains('검색'), '유입채널 대분류'] = '검색'

In [ ]:
#잠시 파악하기 쉽게 변수를 한정하겠습니다.
df = df[['주문접수일시', '주문번호', '유입채널 대분류', '시간대 대분류', '주문월', '아이디', '판매자상품명', '판매가', '표준카테고리']]

In [ ]:
df.head()

In [ ]:
#월별로 데이터 나누기
df_1 = df[df['주문월'] == 1]
df_2 = df[df['주문월'] == 2]
df_3 = df[df['주문월'] == 3]
df_4 = df[df['주문월'] == 4]
df_5 = df[df['주문월'] == 5]
df_6 = df[df['주문월'] == 6]
df_7 = df[df['주문월'] == 7]
df_8 = df[df['주문월'] == 8]
df_9 = df[df['주문월'] == 9]

## 1) 1월 분석하기

In [ ]:
df_1['판매자상품명'].value_counts().head(10)
sns.countplot(y='판매자상품명', data=df_1, order=[df_1['판매자상품명'].value_counts().head(10).index[i] for i in range(0,10)])

In [ ]:
plt.figure(figsize=(6,3))
sns.countplot(y='표준카테고리', data=df_1, order=[df_1['표준카테고리'].value_counts().head(5).index[i] for i in range(0,5)])

-> 1월 : 만두 > 선물세트 > 즉석밥 > 국/탕 > 스테이크, 햄 등등
* 아무래도 설날 준비를 이유로 만두와 선물세트의 구매율이 많은 것으로 보입니다.
* 2022년 설 1~2주전에 만두와 선물세트에 대한 광고와 할인을 하는 실시하는 것을 제안합니다.

## 2) 2월 분석하기

In [ ]:
df_2['판매자상품명'].value_counts().head(10)
sns.countplot(y='판매자상품명', data=df_2, order=[df_2['판매자상품명'].value_counts().head(10).index[i] for i in range(0,10)])

In [ ]:
plt.figure(figsize=(6,5))
sns.countplot(y='표준카테고리', data=df_2, order=[df_2['표준카테고리'].value_counts().head(10).index[i] for i in range(0,10)])

-> 2월 : 만두 > 요리소스 > 즉석밥 > 국/탕 > 스테이크, 햄 등등
* 만두의 비율이 압도적으로 많으므로 할인을 실시했는지 원인 살펴보겠습니다.(코드 삽입)

## 2-1) 만두 할인 분석 코드

In [ ]:
# 2월에 가장 많이 팔린 만두 '판매자상품명' 10개 출력

df_2[df_2['표준카테고리'] == '냉장/냉동/간편식 > 떡볶이/만두 > 만두']['판매자상품명'].value_counts().head(10).index

In [ ]:
df_2_mandu = df_2[df_2['표준카테고리'] == '냉장/냉동/간편식 > 떡볶이/만두 > 만두'].groupby('판매자상품명')[['판매가']].mean()
df_2_mandu.head()

In [ ]:
df_mandu = df[(df['표준카테고리'] == '냉장/냉동/간편식 > 떡볶이/만두 > 만두') & (df['주문월'] !=2)].groupby('판매자상품명')[['판매가']].mean() 
df_mandu.head()

In [ ]:
df_mandu_c = df_2[df_2['표준카테고리'] == '냉장/냉동/간편식 > 떡볶이/만두 > 만두'].groupby('판매자상품명')[['판매가']].count()
df_mandu_c.head()

In [ ]:
df_man = pd.merge(df_2_mandu, df_mandu_c, on='판매자상품명')

In [ ]:
df_man1 = pd.merge(df_man, df_mandu, on='판매자상품명')
df_man1.rename(columns={'판매가_x':'2월 평균단가', '판매가_y' : '2월 주문수', '판매가':'타월 평균단가'}, inplace=True)
df_man1.head()

In [ ]:
df_man1['평균 할인율'] = ((df_man1['2월 평균단가'] / df_man1['타월 평균단가']) - 1) * 100

In [ ]:
df_man1.sort_values(by='2월 주문수', ascending=False).head(10)

'[특가+증정] CJ 비비고 왕교자 1.05kg X 3개 + 찐만두 증정' 상품의 할인이 2월의 만두 매출에 영향이었을 것이며,
타월의 평균단가보다 비싼 상품은, 2월 이후에 더 싼 가격에 판매하여, 매출을 올렸을 것으로 판단된다.

## 3) 3월 분석하기

In [ ]:
df_3['판매자상품명'].value_counts().head(10)
sns.countplot(y='판매자상품명', data=df_3, order=[df_3['판매자상품명'].value_counts().head(10).index[i] for i in range(0,10)])

In [ ]:
plt.figure(figsize=(6,5))
sns.countplot(y='표준카테고리', data=df_3, order=[df_3['표준카테고리'].value_counts().head(10).index[i] for i in range(0,10)])

3월은 만두 > 조미료 > 국/탕 > 즉석밥/죽 > 돈가스

## 4) 4월 분석하기

In [ ]:
df_4['판매자상품명'].value_counts().head(10)
sns.countplot(y='판매자상품명', data=df_4, order=[df_4['판매자상품명'].value_counts().head(10).index[i] for i in range(0,10)])

In [ ]:
plt.figure(figsize=(6,5))
sns.countplot(y='표준카테고리', data=df_4, order=[df_4['표준카테고리'].value_counts().head(10).index[i] for i in range(0,10)])

4월 - 만두 > 즉석밥 > 장류 > 돈가스 > 식용유

## 5) 5월 분석하기

In [ ]:
df_5['판매자상품명'].value_counts().head(10)
sns.countplot(y='판매자상품명', data=df_5, order=[df_5['판매자상품명'].value_counts().head(10).index[i] for i in range(0,10)])

In [ ]:
plt.figure(figsize=(6,5))
sns.countplot(y='표준카테고리', data=df_5, order=[df_5['표준카테고리'].value_counts().head(10).index[i] for i in range(0,10)])

5월: 식용유 > 만두 > 즉석밥 > 젤리/푸딩
* 식용유 할인했는지 유입채널은 무엇인지 살펴보겠습니다.(코드 삽입)

### 5-1) 식용유 매출의 원인 분석하기

In [ ]:
# 포도씨유 900ML * 2개 상품만을 차지하고 있어, 카테고리에 해당되는 상품별로 가격을 비교할 필요는 없어보입니다.

df_5[df_5['표준카테고리'] == '가공식품 > 식용유/오일 > 식용유/오일선물세트']['판매자상품명'].value_counts().index

In [ ]:
# 5월달의 식용유 가격대, 평균가, 최빈가 / # 타월의 식용유 가격대, 평균가, 최빈가

print(df_5[df_5['판매자상품명'] == '[CJ직배송]포도씨유 900ML X 2개']['판매가'].unique())
print(df_5[df_5['판매자상품명'] == '[CJ직배송]포도씨유 900ML X 2개']['판매가'].mean())
print(df_5[df_5['판매자상품명'] == '[CJ직배송]포도씨유 900ML X 2개']['판매가'].mode())
print('---------------------------------------------')
print(df[(df['판매자상품명'] == '[CJ직배송]포도씨유 900ML X 2개') & (df['주문월'] != 5)]['판매가'].unique())
print(df[(df['판매자상품명'] == '[CJ직배송]포도씨유 900ML X 2개') & (df['주문월'] != 5)]['판매가'].mean())
print(df[(df['판매자상품명'] == '[CJ직배송]포도씨유 900ML X 2개') & (df['주문월'] != 5)]['판매가'].mode())


In [ ]:
df_5[df_5['판매자상품명'] == '[CJ직배송]포도씨유 900ML X 2개']['유입채널 대분류'].value_counts()

* 5월에는 할인이 매출 상승의 원인으로 파악할 수 있을 것이며, 모바일 공유/ 가격비교를 통해 많이 유입했음을 알 수 있습니다.
* 할인 행사에 대한 블로그 홍보글 등의 입소문을 타고 많이 유입했을 것으로 판단됩니다.

## 6) 6월 분석하기

In [ ]:
df_6['판매자상품명'].value_counts().head(10)
sns.countplot(y='판매자상품명', data=df_6, order=[df_6['판매자상품명'].value_counts().head(10).index[i] for i in range(0,10)])

In [ ]:
plt.figure(figsize=(6,5))
sns.countplot(y='표준카테고리', data=df_6, order=[df_6['표준카테고리'].value_counts().head(10).index[i] for i in range(0,10)])

6월 : 즉석밥 > 국/탕 > 볶음밥 > 만두 > 돈가스
* 즉석밥, 곰탕의 비율이 높다. 할인했는지, 유입채널은 무엇인지 살펴보겠습니다.(코드삽입)

### 6-1) 즉석밥의 매출이 높은 원인 분석하기

In [ ]:
df_6[df_6['표준카테고리'] == '냉장/냉동/간편식 > 즉석밥/죽/스프 > 즉석밥']['판매자상품명'].value_counts().head(10).index

In [ ]:
df_6_rice = df_6[df_6['표준카테고리'] == '냉장/냉동/간편식 > 즉석밥/죽/스프 > 즉석밥'].groupby('판매자상품명')[['판매가']].mean()
df_6_rice

In [ ]:
df_6_ricec = df_6[df_6['표준카테고리'] == '냉장/냉동/간편식 > 즉석밥/죽/스프 > 즉석밥'].groupby('판매자상품명')[['판매가']].count()
df_6_ricec

In [ ]:
df_6_rice1 = pd.merge(df_6_rice, df_6_ricec, on='판매자상품명')
df_6_rice1

In [ ]:
df_n6_rice = df[(df['표준카테고리'] == '냉장/냉동/간편식 > 즉석밥/죽/스프 > 즉석밥') & (df['주문월'] !=6)].groupby('판매자상품명')[['판매가']].mean() 
df_n6_rice

In [ ]:
df_6_rice = pd.merge(df_6_rice1, df_n6_rice, on='판매자상품명')
df_6_rice.rename(columns={'판매가_x':'6월 평균단가', '판매가_y' : '6월 주문수', '판매가':'타월 평균단가'}, inplace=True)
df_6_rice.head()

In [ ]:
df_6_rice['평균 할인율'] = ((df_6_rice['6월 평균단가'] / df_6_rice['타월 평균단가']) - 1) * 100

In [ ]:
df_6_rice.sort_values(by='6월 주문수', ascending=False).head(10)

In [ ]:
df_6[df_6['표준카테고리'] == '냉장/냉동/간편식 > 국/탕/찌개 > 국/탕']['판매자상품명'].value_counts().head(10).index

In [ ]:
df_6_gook = df_6[df_6['표준카테고리'] == '냉장/냉동/간편식 > 국/탕/찌개 > 국/탕'].groupby('판매자상품명')[['판매가']].mean()
df_6_gookc = df_6[df_6['표준카테고리'] == '냉장/냉동/간편식 > 국/탕/찌개 > 국/탕'].groupby('판매자상품명')[['판매가']].count()
df_6_gook1 = pd.merge(df_6_gook, df_6_gookc, on='판매자상품명')
df_n6_gook = df[(df['표준카테고리'] == '냉장/냉동/간편식 > 국/탕/찌개 > 국/탕') & (df['주문월'] !=6)].groupby('판매자상품명')[['판매가']].mean()
df_6_gook = pd.merge(df_6_gook1, df_n6_gook, on='판매자상품명')
df_6_gook.rename(columns={'판매가_x':'6월 평균단가', '판매가_y' : '6월 주문수', '판매가':'타월 평균단가'}, inplace=True)
df_6_gook['평균 할인율'] = ((df_6_gook['6월 평균단가'] / df_6_gook['타월 평균단가']) - 1) * 100
df_6_gook.sort_values(by='6월 주문수', ascending=False).head(10)

In [ ]:
df_6[df_6['표준카테고리'] == '냉장/냉동/간편식 > 즉석밥/죽/스프 > 즉석밥']['유입채널 대분류'].value_counts()

In [ ]:
df_6[df_6['표준카테고리'] == '냉장/냉동/간편식 > 국/탕/찌개 > 국/탕']['유입채널 대분류'].value_counts()

-> 두 카테고리 모두, CJ 앱/사이트 내에서 유입하여 주문한 경우가 대부분으로, 앱 내에서의 광고 효과가 많은 영향을 끼친 것 같습니다.

국/ 꼬리곰탕의 경우에는 할인 행사가 매출에 중대한 영향을 끼친 것으로 판단되며,

즉석밥의 경우에는 증정행사 및 LPoint 지급 행사가 중대한 영향을 끼친 것으로 판단할 수 있을 것입니다.

## 7) 7월 분석하기

In [ ]:
df_7['판매자상품명'].value_counts().head(10)
sns.countplot(y='판매자상품명', data=df_7, order=[df_7['판매자상품명'].value_counts().head(10).index[i] for i in range(0,10)])

In [ ]:
plt.figure(figsize=(6,5))
sns.countplot(y='표준카테고리', data=df_7, order=[df_7['표준카테고리'].value_counts().head(10).index[i] for i in range(0,10)])

7월: 즉석밥 > 푸딩 > 면 > 소스 > 국/탕 (즉석밥 행사 여전, 푸딩 행사 시작)
행사실시 코드 삽입

## 7-1) 푸딩의 매출 폭등 원인 분석하기

In [ ]:
# 6월의 젤리/푸딩 주문 파악

df_6[df_6['표준카테고리'] == '과자/간식 > 캔디/껌/캐러멜 > 젤리/푸딩'].groupby(['판매자상품명', '판매가'])[['주문번호']].count().sort_values(by='주문번호', ascending=False).head(10)

In [ ]:
# 7월의 젤리/푸딩 주문 파악

df_7[df_7['표준카테고리'] == '과자/간식 > 캔디/껌/캐러멜 > 젤리/푸딩'].groupby(['판매자상품명', '판매가'])[['주문번호']].count().sort_values(by='주문번호', ascending=False).head(10)

In [ ]:
# 묶음 단위 행사로서, 가격대 비교는 의미가 없을듯 하여 삭제하여도 좋을 것 같습니다.

df_7_jel = df_7[df_7['표준카테고리'] == '과자/간식 > 캔디/껌/캐러멜 > 젤리/푸딩'].groupby('판매자상품명')[['판매가']].mean()
df_7_jelc = df_7[df_7['표준카테고리'] == '과자/간식 > 캔디/껌/캐러멜 > 젤리/푸딩'].groupby('판매자상품명')[['판매가']].count()
df_7_jel1 = pd.merge(df_7_jel, df_7_jelc, on='판매자상품명')
df_n7_jel = df[(df['표준카테고리'] == '과자/간식 > 캔디/껌/캐러멜 > 젤리/푸딩') & (df['주문월'] !=7)].groupby('판매자상품명')[['판매가']].mean()
df_7_jel = pd.merge(df_7_jel1, df_n7_jel, on='판매자상품명')
df_7_jel.rename(columns={'판매가_x':'7월 평균단가', '판매가_y' : '7월 주문수', '판매가':'타월 평균단가'}, inplace=True)
df_7_jel['평균 할인율'] = ((df_7_jel['7월 평균단가'] / df_7_jel['타월 평균단가']) - 1) * 100
df_7_jel[df_7_jel['평균 할인율'] !=0].sort_values(by='7월 주문수', ascending=False).head(10)

계절성 상품인 물젤리 상품을 묶음 단위로 할인하여 판매하였으며, 

6월까지만 해도 210g짜리 10개씩 팔던 상품을 조금 더 싼 가격으로 7월에 90g,130ml짜리 30개 묶음으로 판매하였더니 주문량이 폭발적으로 증가한 것을 알 수 있다. 

물젤리 상품은, 적은 부피의 많은 양을 묶음으로 팔아야 주문이 폭발적으로 증가하는 경향이 있다고 파악할 수 있겠습니다.

## 8) 8월 분석하기

In [ ]:
df_8['판매자상품명'].value_counts().head(10)
sns.countplot(y='판매자상품명', data=df_8, order=[df_8['판매자상품명'].value_counts().head(10).index[i] for i in range(0,10)])

In [ ]:
plt.figure(figsize=(6,5))
sns.countplot(y='표준카테고리', data=df_8, order=[df_8['표준카테고리'].value_counts().head(10).index[i] for i in range(0,10)])

8월 : 즉석밥 > 국/탕 > 햄통조림 > 만두 > 식용유 (즉석밥 여러 행사 진행중, 국/탕 및 통조림 행사가 매출에 큰 영향을 주는걸로 보입니다.)
코드 삽입해보겠습니다.

## 8-1) 국/탕, 햄통조림의 매출 7월 vs 8월 분석

In [ ]:
# 7월의 국/탕 주문 파악

df_7[df_7['표준카테고리'] == '냉장/냉동/간편식 > 국/탕/찌개 > 국/탕'].groupby(['판매자상품명', '판매가'])[['주문번호']].count().sort_values(by='주문번호', ascending=False).head(10)

In [ ]:
# 8월의 국/탕 주문 파악

df_8[df_8['표준카테고리'] == '냉장/냉동/간편식 > 국/탕/찌개 > 국/탕'].groupby(['판매자상품명', '판매가'])[['주문번호']].count().sort_values(by='주문번호', ascending=False).head(10)

In [ ]:
# 7월의 햄통조림 주문 파악

df_7[df_7['표준카테고리'] == '가공식품 > 통조림/캔 > 햄통조림'].groupby(['판매자상품명', '판매가'])[['주문번호']].count().sort_values(by='주문번호', ascending=False).head(10)

In [ ]:
# 8월의 햄통조림 주문 파악

df_8[df_8['표준카테고리'] == '가공식품 > 통조림/캔 > 햄통조림'].groupby(['판매자상품명', '판매가'])[['주문번호']].count().sort_values(by='주문번호', ascending=False).head(10)

* 국/탕의 인기상품은 7월과 대체로 비슷하지만, 8월에 폭발적으로 증가한 것을 보면, 홍보 효과 및 증정품 이벤트 효과인 것 같다.
* 햄통조림은 8월에 20000원 4개 골라담기 행사를 시행하였고, 이로 인한 입소문이 빠르게 번진 것으로 파악된다.

## 9) 9월 분석하기

In [ ]:
df_9['판매자상품명'].value_counts().head(10)
sns.countplot(y='판매자상품명', data=df_9, order=[df_9['판매자상품명'].value_counts().head(10).index[i] for i in range(0,10)])

In [ ]:
plt.figure(figsize=(6,5))
sns.countplot(y='표준카테고리', data=df_9, order=[df_9['표준카테고리'].value_counts().head(10).index[i] for i in range(0,10)])

In [ ]:
import matplotlib.gridspec as gridspec
grid = gridspec.GridSpec(5,2) # 그래프를 9개를 그려보겠습니다.

plt.figure(figsize=(15,30))  
plt.subplots_adjust(wspace=0.5, hspace=0.3) 

months = [i for i in range(1, 10)]

for idx, month in enumerate(months):
    ax = plt.subplot(grid[idx])
    
    sns.countplot(y='표준카테고리',
                data=df[df['주문월'] == month],
                order=[df[df['주문월'] == month]['표준카테고리'].value_counts().head(10).index[i] for i in range(0,10)],
                palette='pastel',
                ax=ax
                )
    ax.set_title(f'Month {month}')

* 월별로 수요가 많은 상품에 대해 할인, 행사한 상품의 가격을 살펴보고 재무 상황에 맞추어 할인 주기적으로 진행하길 제안합니다.
* 월별로 수요가 많은 상품의 유입채널을 살펴보아 광고 활성화하길 제안합니다.


# 상품별 마케팅 방안 고안해보기

## 상관관계를 통해 분석하기

In [ ]:
df.groupby(['주문월', '표준카테고리'])['주문접수일시'].count()

In [ ]:
df_tops = pd.DataFrame(data = [[0] * 9 for i in range(65)], 
                       columns=['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월'], 
                       index=df['표준카테고리'].value_counts().sort_index().index )
df_tops['1월'] = df_1['표준카테고리'].value_counts().sort_values()
df_tops['2월'] = df_2['표준카테고리'].value_counts().sort_values()
df_tops['3월'] = df_3['표준카테고리'].value_counts().sort_values()
df_tops['4월'] = df_4['표준카테고리'].value_counts().sort_values()
df_tops['5월'] = df_5['표준카테고리'].value_counts().sort_values()
df_tops['6월'] = df_6['표준카테고리'].value_counts().sort_values()
df_tops['7월'] = df_7['표준카테고리'].value_counts().sort_values()
df_tops['8월'] = df_8['표준카테고리'].value_counts().sort_values()
df_tops['9월'] = df_9['표준카테고리'].value_counts().sort_values()

df_tops = df_tops.fillna(0)

In [ ]:
df_tops

In [ ]:
df_topT = df_tops.transpose()

In [ ]:
df_topT

In [ ]:
df_topT.corr().head()

In [ ]:
df_topT.corr()['냉장/냉동/간편식 > 떡볶이/만두 > 만두'].sort_values(ascending=False)[1:6]

In [ ]:
df_topT.corr()['냉장/냉동/간편식 > 떡볶이/만두 > 만두'].sort_values(ascending=False).index[1:6]

In [ ]:
sns.barplot(y=df_topT.corr()['냉장/냉동/간편식 > 떡볶이/만두 > 만두'].sort_values(ascending=False).index[1:6],
            x=df_topT.corr()['냉장/냉동/간편식 > 떡볶이/만두 > 만두'].sort_values(ascending=False)[1:6])

In [ ]:
df_topT.corr()['가공식품 > 소스/드레싱 > 굴소스/중화요리소스'].sort_values(ascending=False)

In [ ]:
sns.barplot(y=df_topT.corr()['가공식품 > 소스/드레싱 > 굴소스/중화요리소스'].sort_values(ascending=False).index[1:6],
            x=df_topT.corr()['가공식품 > 소스/드레싱 > 굴소스/중화요리소스'].sort_values(ascending=False)[1:6])

In [ ]:
df_topT.corr()['가공식품 > 식용유/오일 > 식용유/오일선물세트'].sort_values(ascending=False)

In [ ]:
sns.barplot(y=df_topT.corr()['가공식품 > 식용유/오일 > 식용유/오일선물세트'].sort_values(ascending=False).index[1:6],
            x=df_topT.corr()['가공식품 > 식용유/오일 > 식용유/오일선물세트'].sort_values(ascending=False)[1:6])

In [ ]:
df_topT.corr()['가공식품 > 장류 > 고추장/초고추장'].sort_values(ascending=False)

In [ ]:
sns.barplot(y=df_topT.corr()['가공식품 > 장류 > 고추장/초고추장'].sort_values(ascending=False).index[1:6],
            x=df_topT.corr()['가공식품 > 장류 > 고추장/초고추장'].sort_values(ascending=False)[1:6])

## 상품별 마케팅 제안

* 각 상품별로 분석

* 1. 만두, 즉석밥, 국/탕, 돈가스 - 매월 상위권 차트에 있으며, 계절에 상관없이 많이 사는 것으로 파악됩니다. 계절 상관없이, 이벤트를 진행한다면, 매출이 많이 오를 가능성이 보입니다.

* 2. 캔/통조림 선물세트 - 주로 설/추석 기간에 선물세트로 주문 상위권에 올라오는 것 같으며, 명절 기간에 홍보, 할인 이벤트를 집중적으로 하는걸 제안합니다..

* 3. 젤리/푸딩 - 유독 여름에만 매출이 많은 상품으로, 여름에 홍보 효과를 강화하길 제안합니다.

* 4. 식용유 및 고추장 - 각자 상관관계가 높은 상품과 함께 이벤트 진행하길 제안합니다.(예를 들어 상관관계가 0.77이상인 식용유와 부침가루와 함께 묶어 행사 진행, 상관관계가 0.95이상인 고추장과 즉석밥을 묶어 행사 진행)

# 3. 월별로 매출과 판매량이 가장 많은 날짜를 중점으로 분석하기

## 1) 필요한 변수 생성하기

In [ ]:
df = pd.read_csv(path+'CJ제일제당 주문 데이터(전처리).csv')

In [ ]:
df['유입채널 대분류'] = '기타 및 이벤트 광고'
df.loc[((df['유입채널'] == '유실 채널상세') | (df['유입채널'] == '디폴트 채널상세')), '유입채널 대분류'] = 'CJ 앱/사이트'
df.loc[df['유입채널'].str.contains('가격비교'), '유입채널 대분류'] = '가격비교'
df.loc[df['유입채널'].str.contains('모바일'), '유입채널 대분류'] = '모바일 공유'
df.loc[df['유입채널'].str.contains('검색'), '유입채널 대분류'] = '검색'

In [ ]:
#주문 시간빼고 날짜만 있는 변수 생성
df['주문날짜'] = df['주문번호']//(10**8)

In [ ]:
df['주문날짜'].astype(str)

In [ ]:
df['주문날짜'] = pd.to_datetime(df['주문날짜'], format = "%Y%m%d")

In [ ]:
df['주문날짜'].head()

In [ ]:
#일별 파악을 위해 변수 생성
df['주문day'] = df['주문날짜'].dt.day

In [ ]:
df['주문day']

In [ ]:
#판매량이 많은 순서를 알아보기
df.groupby('주문날짜')['주문번호'].count()

## 2) 월별로 가장 많은 판매량이 나온 날짜 그래프를 통해 분석하기

In [ ]:
import matplotlib.gridspec as gridspec
grid = gridspec.GridSpec(3,3) # 그래프를 9개를 그려보겠습니다.

plt.figure(figsize=(15,15))  
plt.subplots_adjust(wspace=0.5, hspace=0.3) 

months = [i for i in range(1, 10)]

for idx, month in enumerate(months):
    ax = plt.subplot(grid[idx])
    
    sns.histplot(x='주문day',
                 data=df[df['주문월'] == month],
                 palette='pastel',
                 bins=max(df['주문day']),
                 ax=ax
                )
    ax.set_title(f'Month {month}')

- 판매량이 많은 날짜
(-주변 날짜보다 2~3배 이상 눈에 띄게 큰 날짜를 중점적으로 파악하였습니다. -2월은 판매량 자체가 너무 적으므로 생략하였습니다.)
* 1월 - 9일
* 3월 - 13, 14일, 21일
* 4월 - 24일
* 5월 - 16일, 25일 (2천건 이상)
* 6월 - 8일, 9일(3천건), 12일, 25일
* 7월 - 4일, 13일, 21일, 27일, 29일
* 8월 - 3일, 6일, 7일(가장 많음, 7천건 이상), 17일, 31일에도 유독 많음
* 9월 - 19일 (상대적으로 다른 달에 비해 너무 적지만 그나마 많은 날짜)

## 3) 월별로 가장 많은 판매액(매출)이 나온 날짜 그래프를 통해 분석하기

In [ ]:
import matplotlib.gridspec as gridspec
grid = gridspec.GridSpec(5,2) # 그래프를 9개를 그려보겠습니다.

plt.figure(figsize=(15,30))  
plt.subplots_adjust(wspace=0.5, hspace=0.3) 

months = [i for i in range(1, 10)]

for idx, month in enumerate(months):
    ax = plt.subplot(grid[idx])
    
    sns.barplot(x='주문day',
                y='판매액',
                estimator=sum,
                ci=None,
                data=df[df['주문월'] == month],
                palette='pastel',
                ax=ax
                )
    ax.set_title(f'Month {month}')

* 매출이 큰 날짜 
(-상대적으로 매출이 적은 2월은 제외하였습니다.
-또한 9월은 판매량이 적고 매출도 눈에 띄게 많이 나온 날짜가 없어 제외하였습니다.)
* 1월 - 9일, 24일
* 3월 - 13, 14일, 21일
* 4월 - 13, 24일
* 5월 - 16일, 25일
* 6월 - 8일, 9일(3천건), 12일, 25일
* 7월 - 4일, 13일, 21일, 27일, 29일
* 8월 - 3일, 6일, 7일(가장 많음, 7천건 이상), 17, 25, 31일에도 유독 많았습니다.

* 주문건수가 많은 날은 대체로 매출도 높은 경향이 있습니다.
* 특히 주문건수, 매출이 모두 높은 날짜

* 1월 9, 24일 
* 3월 13, 14, 21일, 
* 4월 24일, 
* 5월 16, 25일 (5월부터 기하급수적으로 늘어나기 시작) 
* 6월 8,9,11, 25일, 
* 7월 4, 13, 21, 27, 29일, 
* 8월 3, 6, 7일

### 3-1) 1월 데이터 (9, 24일)

In [ ]:
# 1월 데이터 (9, 24일 살펴보기)

df0109 = df[df['주문날짜'] == '2022-01-09']
df0124 = df[df['주문날짜'] == '2022-01-24']

9일부터 살펴보겠습니다.

In [ ]:
# 왕교자, 사골곰탕, 짬뽕 등이 이날 유독 많이 판매되었습니다.
df0109['판매자상품명'].value_counts().head(10)

In [ ]:
df0109['유입채널 대분류'].value_counts()

In [ ]:
df0109.groupby('판매자상품명')['판매액'].sum().sort_values(ascending=False)

In [ ]:
print(df[df['판매자상품명'] == '[CJ직배송]CJ비비고왕교자1.05KG X 2개']['판매가'].unique())
print(df[df['판매자상품명'] == '[CJ직배송][22년 설 선물세트]21년 설 비비고 토종김 5호']['판매가'].unique())

In [ ]:
# 9일을 살펴보니 할인을 진행했다는 것을 알 수 있습니다.

print(df0109[df0109['판매자상품명'] == '[CJ직배송]CJ비비고왕교자1.05KG X 2개']['판매가'].unique())
print(df0109[df0109['판매자상품명'] == '[CJ직배송][22년 설 선물세트]21년 설 비비고 토종김 5호']['판매가'].unique())

In [ ]:
# 이건 매출액이 커서 살펴보았더니 주문 한 건에서 250개를 사갔습니다.
# 이건 순간적인 효과라서 중요한 변수로 보긴 어려울 것 같습니다.
# 수량보다는 주문 건수가 중요한 변수인 것으로 보입니다.

df0109[df0109['판매자상품명'] == '[CJ직배송][22년 설 선물세트]21년 설 비비고 토종김 5호']['수량']

In [ ]:
## 1월 9일날은 가격비교 사이트가 압도적으로 높았습니다.

df0109.groupby('유입채널 대분류')['판매액'].sum().sort_values(ascending=False)

In [ ]:
## 1월 9일날은 가격비교 사이트가 압도적으로 높았습니다.

df0109.groupby('유입채널')['판매액'].sum().sort_values(ascending=False)

1월 설에는 만두나 설 선물세트 할인행사를 활성화하고 가격비교 사이트에 광고를 싣는 것을 제안합니다.
또한 CJ/앱 푸시알림을 전 고객에게 활성화하는 것을 제안합니다.

24일 살펴보겠습니다.

In [ ]:
# 1/24날은 설 선물 세트로 매출을 올린 것 같아 보입니다.
df0124.groupby('판매자상품명')['판매액'].sum().sort_values(ascending=False).head(10)

In [ ]:
# 하지만 1/24날 가장 많은 주문이 들어온 상품은 주문건수가 극단적으로 많지 않습니다.
print(df0124[df0124['판매자상품명'] == '[CJ직배송][22년 설 선물세트]스팸3호']['유입채널 대분류'].value_counts())
print('\n')
print(df0124[df0124['판매자상품명'] == '[CJ직배송]특별한선택 Y호']['유입채널 대분류'].value_counts())
print('\n')
print(df0124[df0124['판매자상품명'] == '[CJ직배송]특별한선택 Y호']['수량'].unique())

In [ ]:
# 1월 9일날은 가격비교 사이트 9백 만원의 매출을 거두었는데, 이 날은 123만원을 거두었습니다.
df0124.groupby('유입채널 대분류')['판매액'].sum().sort_values(ascending=False)

* 1월 9일 판매 데이터는 네이버를 통한 PCS 가격비교_롯데ON이 142건이나 되는데, 1월 24일 판매 데이터는 디폴트 채널에서만 거의 판매되었습니다. 가격에 대한 메리트가 크지 않아 그런 것이라 생각되지만 설 선물세트는 매출을 크게 차지하는 만큼 광고를 활성화하면 좋을 것을 생각됩니다.

### 3-2) 3월 데이터 (13~14, 21일)

In [ ]:
df0313 = df[(df['주문날짜'] == '2022-03-13') | (df['주문날짜'] == '2022-03-14')]
df0321 = df[df['주문날짜'] == '2022-03-21']

13일부터 살펴보겠습니다.

In [ ]:
# 이 날은 CJ 만두, 비비고 식품 등이 많이 팔렸습니다.
df0313['판매자상품명'].value_counts()

In [ ]:
# 가격비교를 통해 약 300~400건, CJ 앱/사이트를 통해 약 500건 정도 주문이 들어왔습니다.
df0313['유입채널 대분류'].value_counts()

In [ ]:
# 이날은 만두가 매출이 무려 1187만원이나 될어 원인을 분석해보았습니다.
df0313.groupby('판매자상품명')['판매액'].sum().sort_values(ascending=False).head(10)

In [ ]:
# 할인도 하지 않았습니다.
print(df0313[df0313['판매자상품명'] == '[CJ직배송][CJ] 비비고 왕교자1.05KG X 3봉 + 수제고기만두 증정']['판매가'].unique())
print(df[df['판매자상품명'] == '[CJ직배송][CJ] 비비고 왕교자1.05KG X 3봉 + 수제고기만두 증정']['판매가'].unique())

In [ ]:
# 수량에 이상치가 있나 확인해보아도 다들 1개나 2개씩 밖에 주문하지 않았습니다.
df0313[df0313['판매자상품명'] == '[CJ직배송][CJ] 비비고 왕교자1.05KG X 3봉 + 수제고기만두 증정']['수량'].unique()

In [ ]:
# 네이버가 엄청난 매출을 올리는데 도와준 것 같습니다. 원인은 유입매체였습니다.
df0313[df0313['판매자상품명'] == '[CJ직배송][CJ] 비비고 왕교자1.05KG X 3봉 + 수제고기만두 증정']['유입채널'].value_counts()

21일 살펴보겠습니다.

In [ ]:
df0321['판매자상품명'].value_counts()

In [ ]:
# 3/13과는 다르게 이번에는 네이버가 원인이지 않습니다.
df0321['유입채널 대분류'].value_counts()

In [ ]:
# 이번에도 만두의 비율이 매출에 영향을 많이 끼쳤습니다.
df0321.groupby('판매자상품명')['판매액'].sum().sort_values(ascending=False).head(10)

In [ ]:
# 원인은 할인이였던 것으로 분석되었습니다.
print(df0321[df0321['판매자상품명'] == '[CJ직배송][CJ] 비비고 왕교자1.05KG X 3봉 + 수제고기만두 증정']['판매가'].unique())
print(df[df['판매자상품명'] == '[CJ직배송][CJ] 비비고 왕교자1.05KG X 3봉 + 수제고기만두 증정']['판매가'].unique())

만두는 네이버에 광고를 지속적으로 띄우거나 할인행사를 진행할 때 앱 푸시 알림을 더욱더 활성화시켜야 하겠습니다.

### 3-3) 4월 데이터 (24일)
* 4/13은 주문 건수가 적은데, 매출만 커서 1/9때 250개 주문한 효과인 것 같아 24일만 분석해보았습니다.

In [ ]:
df0424 = df[df['주문날짜'] == '2022-04-24']

In [ ]:
# 천일염/ 곰탕 특가, 포도씨유의 주문건수가 높습니다.
df0424['판매자상품명'].value_counts()

In [ ]:
# 많이 팔리고, 매출도 큰 포도씨유를 집중 분석하겠습니다.
df0424.groupby('판매자상품명')['판매액'].sum().sort_values(ascending=False).head(10)

In [ ]:
# 천일염/ 곰탕은 특가여서 건수가 많지만 포도씨유야가 높은 원인을 살펴봐야겠습니다.
df0424['판매자상품명'].value_counts()

In [ ]:
# 보통 유입채널로 들어왔습니다.
print(df0424[df0424['판매자상품명'] == '[CJ직배송]포도씨유 900ML X 2개']['유입채널 대분류'].value_counts())

In [ ]:
# 할인의 효과도 아닌 것 같습니다.

print(df0424[df0424['판매자상품명'] == '[CJ직배송]포도씨유 900ML X 2개']['판매가'].unique())
print(df[df['판매자상품명'] == '[CJ직배송]포도씨유 900ML X 2개']['판매가'].unique())
print(df[df['판매자상품명'] == '[CJ직배송]포도씨유 900ML X 2개']['판매가'].mean())

In [ ]:
# 수량에 이상치는 없습니다.
df0424[df0424['판매자상품명'] == '[CJ직배송]포도씨유 900ML X 2개']['수량'].unique()

포도씨유는 주문건수가 높지 않더라도 매출액의 큰 비중을 차지하기 때문에 할인행사를 열거나 광고 빈도를 높이는 것을 제안합니다.

### 3-4) 5월 데이터 (16, 25일)

In [ ]:
df0516 = df[df['주문날짜'] == '2022-05-16']
df0525 = df[df['주문날짜'] == '2022-05-25']

16일부터 살펴보겠습니다.

In [ ]:
# 아까 4월에 등장했던 포도씨유의 수량이 높습니다.
df0516['판매자상품명'].value_counts().head()

In [ ]:
# 모바일에서 홍보를 크게한 모양입니다. 그리고 CJ앱/사이트도 많습니다.
# 이건 모바일 어디에서 홍보글 보고 바로 들어왔거나, 읽고 조금 생각해보다 사야겠다 하고 채널로 직접 들어온 거 같습니다.
# 결론은 블로그나 SNS를 통한 홍보가 큰 역할을 한 것으로 보입니다.
df0516['유입채널 대분류'].value_counts().head()

In [ ]:
# 포도씨유의 매출액이 무려 4천 4백만원이나 됩니다.
df0516.groupby('판매자상품명')['판매액'].sum().sort_values(ascending=False).head(10)

In [ ]:
print(df0516[df0516['판매자상품명'] == '[CJ직배송]포도씨유 900ML X 2개']['판매가'].unique())

In [ ]:
# SNS에서 2만원에 판다는 홍보글에 사람들이 많이 구매한 것으로 보입니다.
df0516[df0516['판매자상품명'] == '[CJ직배송]포도씨유 900ML X 2개'].groupby('유입매체')['판매가'].value_counts().head(20)

25일 살펴보겠습니다.

In [ ]:
# 왕교자는 원래 수요가 좋은데 이날은 5% 할인 소식에 많이 구매하는 것으로 보입니다.
# 갑자기 이날은 죽 골라담기 주문 또한 할인의 영향으로 많이 유입된 것으로 보입니다.

df0525['판매자상품명'].value_counts()

In [ ]:
# 아무래도 할인 쿠폰 제공이니 앱/웹을 통한 유입이 많습니다.
df0525['유입채널 대분류'].value_counts().head()

In [ ]:
# 엄청난 매출을 두 상품이 차지하고 있습니다.
df0525.groupby('판매자상품명')['판매액'].sum().sort_values(ascending=False).head(10)

만두는 원래 수요가 좋고 죽 골라담기 상품은 매출이 매우 높으니 주기적으로 할인을 진행하는 것을 제안합니다.

### 3-5) 6월 데이터 (8, 9, 25일) 

In [ ]:
df0608 = df[(df['주문날짜'] == '2022-06-08') | (df['주문날짜'] == '2022-06-09')]
df0625 = df[df['주문날짜'] == '2022-06-25']

8일 살펴보겠습니다.

In [ ]:
#8일에는 곰탕, 볶음밥 제품이 압도적입니다.
df0608['판매자상품명'].value_counts().head()

In [ ]:
df0608['유입채널 대분류'].value_counts().head()

In [ ]:
df0608.groupby('판매자상품명')['판매액'].sum().sort_values(ascending=False).head(10)

In [ ]:
print(df0608[df0608['판매자상품명'] == '비비고 BEST 볶음밥 3종 특가[불고기 비빔밥 + 매콤부추고기볶음밥 + 간장버터장조림볶음밥]']['판매가'].unique())
print(df[df['판매자상품명'] == '비비고 BEST 볶음밥 3종 특가[불고기 비빔밥 + 매콤부추고기볶음밥 + 간장버터장조림볶음밥]']['판매가'].unique())
print(df[df['판매자상품명'] == '비비고 BEST 볶음밥 3종 특가[불고기 비빔밥 + 매콤부추고기볶음밥 + 간장버터장조림볶음밥]']['판매가'].mean())

In [ ]:
print(df0608[df0608['판매자상품명'] == '비비고 BEST 볶음밥 3종 특가[불고기 비빔밥 + 매콤부추고기볶음밥 + 간장버터장조림볶음밥]']['판매가'].value_counts())

원인은 할인이였고 그렇기에 앱/웹을 통한 유입이 많았더것으로 보입니다.

25일 살펴보겠습니다.

In [ ]:
df0625['판매자상품명'].value_counts().head()

In [ ]:
df0625['유입채널 대분류'].value_counts().head()

In [ ]:
df0625.groupby('판매자상품명')['판매액'].sum().sort_values(ascending=False).head(10)

In [ ]:
print(df0625[df0625['판매자상품명'] == '[Lpoint 1,000P 지급]햇반 작은공기, 100% 현미밥 x 36개 특가 (1,000set 한정)']['판매가'].unique())
print(df[df['판매자상품명'] == '[Lpoint 1,000P 지급]햇반 작은공기, 100% 현미밥 x 36개 특가 (1,000set 한정)']['판매가'].unique())
print(df[df['판매자상품명'] == '[Lpoint 1,000P 지급]햇반 작은공기, 100% 현미밥 x 36개 특가 (1,000set 한정)']['판매가'].mean())

특가 행사로 인하여 매출이 많은 것으로 보입니다. 앱, 웹을 통해 매출이 높은 햇반 종류 할인을 할 때 푸시알림을 활성화하는 것이 좋을 것 같습니다.

### 3-6) 7월 데이터 분석 (4, 13, 21, 27, 29일) 분석

In [ ]:
df0704 = df[df['주문날짜'] == '2022-07-04']
df0713 = df[df['주문날짜'] == '2022-07-13']
df0721 = df[df['주문날짜'] == '2022-07-21']
df0727 = df[df['주문날짜'] == '2022-07-27']
df0729 = df[df['주문날짜'] == '2022-07-29']

#### 3-6-1) 7월 4일

In [ ]:
# 4일에는 현미밥 제품이 압도적으로 많습니다.
df0704['판매자상품명'].value_counts().head()

In [ ]:
df0704['유입채널 대분류'].value_counts().head()

In [ ]:
df0704.groupby('판매자상품명')['판매액'].sum().sort_values(ascending=False).head(10)

In [ ]:
print(df0704[df0704['판매자상품명'] == '햇반 작은공기/100% 현미밥130g x 36개']['판매가'].unique())
print(df[df['판매자상품명'] == '햇반 작은공기/100% 현미밥130g x 36개']['판매가'].unique())
print(df[df['판매자상품명'] == '햇반 작은공기/100% 현미밥130g x 36개']['판매가'].mean())

이 또한, 특가 행사로 인하여 매출이 많은 것으로 보입니다. 앱, 웹을 통해 매출이 높은 햇반 종류 할인을 할 때 푸시알림을 활성화하는 것이 좋을 것 같습니다.

#### 3-6-2) 7월 13일

In [ ]:
# 죽 상품이 압도적으로 많습니다.
df0713['판매자상품명'].value_counts().head()

In [ ]:
df0713.groupby('판매자상품명')['판매액'].sum().sort_values(ascending=False).head(10)

In [ ]:
df0713['유입채널 대분류'].value_counts().head()

In [ ]:
print(df0713[df0713['판매자상품명'] == '[전참시 속 보양식사]누룽지닭백숙죽/전복삼계죽/녹두닭죽 外 + 보양닭백숙죽 증정']['판매가'].unique())
print(df[df['판매자상품명'] == '[전참시 속 보양식사]누룽지닭백숙죽/전복삼계죽/녹두닭죽 外 + 보양닭백숙죽 증정']['판매가'].unique())
print(df[df['판매자상품명'] == '[전참시 속 보양식사]누룽지닭백숙죽/전복삼계죽/녹두닭죽 外 + 보양닭백숙죽 증정']['판매가'].mean())

당일 주문량이 많았던 원인은 할인이였고 그렇기에 앱/웹을 통한 유입이 많았더것으로 보입니다.

21일 살펴보겠습니다.

In [ ]:
# 냉면의 주문이 압도적으로 많습니다.
df0721['판매자상품명'].value_counts().head()

In [ ]:
df0721.groupby('판매자상품명')['판매액'].sum().sort_values(ascending=False).head(10)

In [ ]:
df0721['유입채널 대분류'].value_counts().head()

In [ ]:
print(df0721[df0721['판매자상품명'] == '[CJ직배송]CJ 고메 중화짬뽕 + 중화짜장 + 탕수육 + 깐풍기 세트']['판매가'].unique())
print(df[df['판매자상품명'] == '[CJ직배송]CJ 고메 중화짬뽕 + 중화짜장 + 탕수육 + 깐풍기 세트']['판매가'].unique())
print(df[df['판매자상품명'] == '[CJ직배송]CJ 고메 중화짬뽕 + 중화짜장 + 탕수육 + 깐풍기 세트']['판매가'].mean())

원인은 할인이였고 그렇기에 앱/웹을 통한 유입이 많았더것으로 보입니다.

#### 3-6-3) 7월 27일

In [ ]:
df0727['판매자상품명'].value_counts().head()

In [ ]:
df0727.groupby('판매자상품명')['판매액'].sum().sort_values(ascending=False).head(10)

In [ ]:
df0727['유입채널 대분류'].value_counts().head()

In [ ]:
print(df0727[df0727['판매자상품명'] == '[CJ직배송]쁘띠첼 5종 세트_30개(포도파인애플복숭아밀감 망고)']['판매가'].unique())
print(df[df['판매자상품명'] == '[CJ직배송]쁘띠첼 5종 세트_30개(포도파인애플복숭아밀감 망고)']['판매가'].unique())
print(df[df['판매자상품명'] == '[CJ직배송]쁘띠첼 5종 세트_30개(포도파인애플복숭아밀감 망고)']['판매가'].mean())

특가 행사로 인하여 매출이 많은 것으로 보입니다. 앱, 웹을 통해 특히 여름에 매출이 높은 푸딩(젤리) 종류 할인을 할 때 푸시알림을 활성화하는 것이 좋을 것 같습니다.

#### 3-6-4) 7월 29일

In [ ]:
# 햇반 종류 상품이 매출, 주문량이 압도적으로 많습니다.
df0729['판매자상품명'].value_counts().head()

In [ ]:
df0729.groupby('판매자상품명')['판매액'].sum().sort_values(ascending=False).head(10)

In [ ]:
df0729['유입채널 대분류'].value_counts().head()

In [ ]:
print(df0729[df0729['판매자상품명'] == '[Lpoint&스벅커피쿠폰증정]햇반 솥반 프리미엄 6종 골라담기[택 1]']['판매가'].unique())
print(df[df['판매자상품명'] == '[Lpoint&스벅커피쿠폰증정]햇반 솥반 프리미엄 6종 골라담기[택 1]']['판매가'].unique())
print(df[df['판매자상품명'] == '[Lpoint&스벅커피쿠폰증정]햇반 솥반 프리미엄 6종 골라담기[택 1]']['판매가'].mean())

특가 행사로 인하여 매출이 많은 것으로 보입니다. 앱, 웹을 통해 매출이 높은 햇반 종류 할인을 할 때 푸시알림을 활성화하는 것이 좋을 것 같습니다.

### 3-7) 8월 데이터 (3, 6, 7일)

In [ ]:
df0803 = df[df['주문날짜'] == '2022-08-03']
df0806 = df[df['주문날짜'] == '2022-08-06']
df0807 = df[df['주문날짜'] == '2022-08-07']

#### 3-7-1) 8월 3일

In [ ]:
# 스팸, 만두 상품의 매출, 주문량이 압도적으로 많습니다.
df0803['판매자상품명'].value_counts().head()

In [ ]:
df0803.groupby('판매자상품명')['판매액'].sum().sort_values(ascending=False).head(10)

In [ ]:
df0803.groupby('판매자상품명')['판매액'].sum().sort_values(ascending=False).head(10)

In [ ]:
df0803['유입채널 대분류'].value_counts().head()

In [ ]:
# 원인은 할인이였고 특가 이벤트 행사로 인해 CJ 앱/웹을 통한 유입이 많았더것으로 보입니다.
print(df0803[df0803['판매자상품명'] == 'CJ 스팸 25%라이트/ 스팸클래식 4개 골라담기 [ 택 1]']['판매가'].unique())
print(df[df['판매자상품명'] == 'CJ 스팸 25%라이트/ 스팸클래식 4개 골라담기 [ 택 1]']['판매가'].unique())
print(df[df['판매자상품명'] == 'CJ 스팸 25%라이트/ 스팸클래식 4개 골라담기 [ 택 1]']['판매가'].mean())

#### 3-7-2) 8월 6일 / 8월 7일

In [ ]:
df0806['판매자상품명'].value_counts().head()

In [ ]:
df0806.groupby('판매자상품명')['판매액'].sum().sort_values(ascending=False).head(10)

In [ ]:
df0806['유입채널 대분류'].value_counts().head()

증정행사로 인해 매출이 많은 것으로 보입니다. 앱, 웹을 통해 매출이 높은 국/탕 증정행사를 할 때 푸시알림을 활성화하는 것이 좋을 것 같습니다.

In [ ]:
df0807['판매자상품명'].value_counts().head()

In [ ]:
df0807.groupby('판매자상품명')['판매액'].sum().sort_values(ascending=False).head(10)

In [ ]:
df0807['유입채널 대분류'].value_counts().head()

8/7도 증정 행사로 인하여 매출이 많은 것으로 보입니다. 앱, 웹을 통해 매출이 높은 국/탕, 햇반 골라담기, 기름 등의 증정행사를 할 때 푸시알림을 활성화하는 것이 좋을 것 같습니다.

# 4. 회원별 분석하기

## 1) 분석 용이해지도록 데이터 정리

In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_excel('CJ제일제당 주문 데이터(전처리).xlsx')

In [ ]:
# 아이디는 약 5만개, 주문은 총 9만개가 검색되는 것으로, 회원별 재구매 패턴을 파악하는 것이 중요할 것으로 판단된다.
print(df['아이디'].nunique())
print(len(df))

In [ ]:
df['시간대'] = df['주문접수일시'].dt.hour
df['시간대 대분류'] = '21 ~ 00'
df.loc[(df['시간대'] >= 0) & (df['시간대'] < 3),'시간대 대분류'] = '00 ~ 03'
df.loc[(df['시간대'] >= 3) & (df['시간대'] < 6),'시간대 대분류'] = '03 ~ 06'
df.loc[(df['시간대'] >= 6) & (df['시간대'] < 9),'시간대 대분류'] = '06 ~ 09'
df.loc[(df['시간대'] >= 9) & (df['시간대'] < 12),'시간대 대분류'] = '09 ~ 12'
df.loc[(df['시간대'] >= 12) & (df['시간대'] < 15),'시간대 대분류'] = '12 ~ 15'
df.loc[(df['시간대'] >= 15) & (df['시간대'] < 18),'시간대 대분류'] = '15 ~ 18'
df.loc[(df['시간대'] >= 18) & (df['시간대'] < 21),'시간대 대분류'] = '18 ~ 21'

In [ ]:
df['유입채널 대분류'] = '기타 및 이벤트 광고'
df.loc[((df['유입채널'] == '유실 채널상세') | (df['유입채널'] == '디폴트 채널상세')), '유입채널 대분류'] = 'CJ 앱/사이트'
df.loc[df['유입채널'].str.contains('가격비교'), '유입채널 대분류'] = '가격비교'
df.loc[df['유입채널'].str.contains('모바일'), '유입채널 대분류'] = '모바일 공유'
df.loc[df['유입채널'].str.contains('검색'), '유입채널 대분류'] = '검색'

In [ ]:
df['유입채널 대분류'].value_counts()

In [ ]:
# id 및 유입채널 별로 분석하기 위해, 데이터프레임에서 필요한 변수 위주로 남겨놓습니다.
df = df[['주문접수일시', '주문번호', '유입채널 대분류', '시간대 대분류', '주문월', '아이디', '판매자상품명', '판매가', '표준카테고리']]

* 고안해낸 아이디어로, 각 아이디별로 묶어서 회원의 주문 패턴을 분석하도록 하겠습니다.

* 주문 횟수가 1~2회인 회원의 경우 패턴을 파악하기가 어려우며, 유령회원으로 전환될 가능성이 높다고 판단하여, 
* 주문 횟수가 3회 이상인 회원들에 한하여 회원으로 판단하고, 회원들의 주문 데이터를 분석해보도록 하겠습니다.

In [ ]:
df['아이디'].value_counts().loc[lambda x: x>= 3].index

In [ ]:
# 주문 횟수가 3회 이상인 회원들의 주문 데이터에 한해서 다른 데이터프레임에 저장합니다.

df_id = df[df['아이디'] == df['아이디'].value_counts().loc[lambda x: x>= 3].index[0]]
for i in range(1, len(df['아이디'].value_counts().loc[lambda x: x>= 3].index)):
    df_id = df_id.append(df[df['아이디'] == df['아이디'].value_counts().loc[lambda x: x>= 3].index[i]])
df_id.head()

In [ ]:
# 아이디가  *, **, ***, **** 으로만 나타나거나, null 값인 경우는 같은 회원으로 파악하기가 어려워 삭제하도록 하겠습니다.

df_id = df_id[(df_id['아이디'] != '*') & (df_id['아이디'] != '**') & 
                  (df_id['아이디'] != '***') & (df_id['아이디'] != '****') & (df_id['아이디'] != '')]

In [ ]:
len(df_id) # 34710개의 데이터가 남아있는 것을 확인할 수 있습니다.

In [ ]:
# 데이터프레임을 csv 파일로 따로 저장하도록 하고, 불러오는 작업을 하였습니다.

# df_id.to_csv('CJ 아이디별 조회.csv', encoding='utf-8')

In [ ]:
df_id = pd.read_csv(path+'CJ 아이디별 조회.csv')

In [ ]:
df_id['주문접수일시'] = pd.to_datetime(df_id['주문접수일시'])
df_id['주문날짜'] = df_id['주문번호']//(10**8)
df_id['주문날짜'].astype(str)
df_id['주문날짜'] = pd.to_datetime(df_id['주문날짜'], format = "%Y%m%d")

## 2-1) 주문 횟수가 가장 많은 회원의 데이터(예시)로 자주 주문하는 시간대 파악하기

In [ ]:
df_id_test = df_id.groupby(['아이디']).count()
df_id_cond = df_id_test[df_id_test['주문접수일시'] >= 2]
df_id_cond.sort_values(by='주문접수일시', ascending=False)

In [ ]:
df_mem1 = df_id[df_id['아이디'] == 'love****']
df_mem1

In [ ]:
# 위 회원이 자주 이용하는 시간대 파악하기
df_mem1['시간대 대분류'].value_counts()

In [ ]:
sns.countplot(x='시간대 대분류', data=df_mem1, order=['00 ~ 03', '03 ~ 06', '06 ~ 09', '09 ~ 12', '12 ~ 15', '15 ~ 18', '18 ~ 21', '21 ~ 00'])

In [ ]:
# 위 회원이 자주 이용하는 유입채널 파악하기
df_mem1['유입채널 대분류'].value_counts()

In [ ]:
# 위 회원이 자주 이용하는 CJ 앱/사이트에서 시간대 파악하기
df_mem1_1 = df_mem1[df_mem1['유입채널 대분류'] == 'CJ 앱/사이트']
sns.countplot(x='시간대 대분류', data=df_mem1_1, 
              order=['00 ~ 03', '03 ~ 06', '06 ~ 09', '09 ~ 12', '12 ~ 15', '15 ~ 18', '18 ~ 21', '21 ~ 00'])

In [ ]:
# 위 회원의 가격비교 이용 시간대 파악하기
df_mem1_2 = df_mem1[df_mem1['유입채널 대분류'] == '가격비교']
sns.countplot(x='시간대 대분류', data=df_mem1_2, 
              order=['00 ~ 03', '03 ~ 06', '06 ~ 09', '09 ~ 12', '12 ~ 15', '15 ~ 18', '18 ~ 21', '21 ~ 00'])

* love**** 회원은 전체적으로 주로 21 ~ 03시인 야간 시간대에 많이 주문하며, 
* CJ 채널에서 직접 주문은 21 ~ 03시, 가격 비교로서의 주문은 오후 시간대 12 ~ 00시에 많음을 알 수 있습니다.

## 2-2) 주문 횟수가 가장 많은 회원이 특정 시간에 자주 주문하는 상품 파악하기

In [ ]:
df_mem1_1['판매자상품명'].value_counts().head()

In [ ]:
# 위 회원은 곰탕, 즉석밥, 만두, 식용유 등이 주문의 과반수를 차지하는 것을 알 수 있습니다.

df_mem1_1_top10 = df_mem1_1['판매자상품명'].value_counts().head(10).index

plt.figure(figsize=(12, 5))
sns.countplot(y='판매자상품명', data=df_mem1_1, order=[df_mem1_1_top10[i] for i in range(0,10)])

In [ ]:
# 카테고리별로 많이 사는 제품 시각화
# 같은 상품인데도 제품별로 이름이 조금씩 다르기 때문에, 한 번 더 확인해주겠습니다.
# ex. 앞에 '[2+1]'이나 '[CJ직배송]', 내지는 '*1개, *3개' 처럼 같은 상품인데도 상품명이 다르기 때문에 카테고리로 묶어주겠습니다.

df_mem1_1_top_category = df_mem1_1['표준카테고리'].value_counts().head(5).index
plt.figure(figsize=(12, 10))
sns.countplot(y='표준카테고리', data=df_mem1_1, order=[df_mem1_1_top_category[i] for i in range(0,5)], hue='시간대 대분류')

In [ ]:
df_mem1_2_top10 = df_mem1_2['판매자상품명'].value_counts().head(10).index

plt.figure(figsize=(12, 10))
sns.countplot(y='판매자상품명', data=df_mem1_2, order=[df_mem1_2_top10[i] for i in range(0,10)], hue='시간대 대분류')

* love**** 회원은 주로 즉석밥(볶음밥), 국/탕(곰탕), 만두(교자), 식용유(포도씨유)를 구매하는 것을 알 수 있으며,
* 주문량이 많은 21 ~ 00 시에 국/탕을 살때 만두를 함께 구매하는 것으로도 볼 수 있을 것 같습니다.
* 그가 주로 주문하는 제품(혹은 같은 카테고리 내 판매량 많은 제품)이 할인 기간,
* 국/탕과 만두를 묶음으로 판매하는 할인 이벤트를 할 때
* CJ 앱/사이트에서는 21 ~ 03시에,
* 가격비교 사이트에서는 12 ~ 00시 사이에 광고를 활성화하여 재구매를 유도하면 좋을 것 같습니다.

## 3) 주문 건수가 30회 이상인 회원들의 월별 주문 패턴 분석하기

In [ ]:
df_id

In [ ]:
# 월별 시간대별 주문량 시각화

import matplotlib.gridspec as gridspec
grid = gridspec.GridSpec(3, 3) 

plt.figure(figsize=(20,15))  
plt.subplots_adjust(wspace=0.5, hspace=0.3) 

months = [i for i in range(1, 10)]

for idx, month in enumerate(months):
    ax = plt.subplot(grid[idx])
    
    sns.countplot(x='시간대 대분류',
                data=df[df['주문월'] == month],
                palette='pastel',
                ax=ax,
                order = ['00 ~ 03', '03 ~ 06', '06 ~ 09', '09 ~ 12', '12 ~ 15', '15 ~ 18', '18 ~ 21', '21 ~ 00']
                )
    ax.set_title(f'Month {month}')

In [ ]:
df_id[df_id['주문월'] == 1].groupby('표준카테고리')['주문번호'].count().sort_values(ascending=False)[:10]

In [ ]:
# 카테고리별로 시각화 해보겠습니다.

import matplotlib.gridspec as gridspec
grid = gridspec.GridSpec(5,2)

plt.figure(figsize=(15,30))  
plt.subplots_adjust(wspace=0.5, hspace=0.3) 

months = [i for i in range(1, 10)]

for idx, month in enumerate(months):
    ax = plt.subplot(grid[idx])
    
    sns.countplot(y='표준카테고리',
                data=df_id[df_id['주문월'] == month],
                order=[df_id[df_id['주문월'] == month]['표준카테고리'].value_counts().head(10).index[i] for i in range(0,10)],
                palette='pastel',
                ax=ax
                )
    ax.set_title(f'Month {month}')

In [ ]:
df_id_1 = df_id[df_id['주문월'] == 1]
df_id_2 = df_id[df_id['주문월'] == 2]
df_id_3 = df_id[df_id['주문월'] == 3]
df_id_4 = df_id[df_id['주문월'] == 4]
df_id_5 = df_id[df_id['주문월'] == 5]
df_id_6 = df_id[df_id['주문월'] == 6]
df_id_7 = df_id[df_id['주문월'] == 7]
df_id_8 = df_id[df_id['주문월'] == 8]
df_id_9 = df_id[df_id['주문월'] == 9]

In [ ]:
df_id['표준카테고리'].nunique()

In [ ]:
df_id_tops = pd.DataFrame(data = [[0] * 9 for i in range(52)], 
                       columns=['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월'], 
                       index=df_id['표준카테고리'].value_counts().sort_index().index )
df_id_tops['1월'] = df_id_1['표준카테고리'].value_counts().sort_values()
df_id_tops['2월'] = df_id_2['표준카테고리'].value_counts().sort_values()
df_id_tops['3월'] = df_id_3['표준카테고리'].value_counts().sort_values()
df_id_tops['4월'] = df_id_4['표준카테고리'].value_counts().sort_values()
df_id_tops['5월'] = df_id_5['표준카테고리'].value_counts().sort_values()
df_id_tops['6월'] = df_id_6['표준카테고리'].value_counts().sort_values()
df_id_tops['7월'] = df_id_7['표준카테고리'].value_counts().sort_values()
df_id_tops['8월'] = df_id_8['표준카테고리'].value_counts().sort_values()
df_id_tops['9월'] = df_id_9['표준카테고리'].value_counts().sort_values()

df_id_tops = df_id_tops.fillna(0)

In [ ]:
df_id_tops = df_id_tops.transpose()

In [ ]:
df_id_tops.corr().head()

In [ ]:
sns.barplot(y=df_id_tops.corr()['냉장/냉동/간편식 > 떡볶이/만두 > 만두'].sort_values(ascending=False).index[1:6],
            x=df_id_tops.corr()['냉장/냉동/간편식 > 떡볶이/만두 > 만두'].sort_values(ascending=False)[1:6])

In [ ]:
sns.barplot(y=df_id_tops.corr()['냉장/냉동/간편식 > 즉석밥/죽/스프 > 즉석밥'].sort_values(ascending=False).index[1:6],
            x=df_id_tops.corr()['냉장/냉동/간편식 > 즉석밥/죽/스프 > 즉석밥'].sort_values(ascending=False)[1:6])

In [ ]:
sns.barplot(y=df_id_tops.corr()['냉장/냉동/간편식 > 국/탕/찌개 > 국/탕'].sort_values(ascending=False).index[1:11],
            x=df_id_tops.corr()['냉장/냉동/간편식 > 국/탕/찌개 > 국/탕'].sort_values(ascending=False)[1:11])

In [ ]:
sns.barplot(y=df_id_tops.corr()['가공식품 > 식용유/오일 > 식용유/오일선물세트'].sort_values(ascending=False).index[1:11],
            x=df_id_tops.corr()['가공식품 > 식용유/오일 > 식용유/오일선물세트'].sort_values(ascending=False)[1:11])

전체 주문 데이터와 비교해본 결과, 

7월에 푸딩의 주문 건수는 주문 횟수가 30회 이상인 단골 회원에게 큰 영향이 없었으며,

특가 이벤트 및 증정 행사를 진행함으로서 주문 인기가 많아진 상품의 트렌드를 오래 따라가는 경향이 있습니다.

즉, 기존 회원들의 재구매를 유도하기 위해서는, 주문 인기가 상품을 토대로 꾸준히 할인/이벤트를 진행해야 합니다.